In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym

Let $N \in \mathbb{N} $ , consider the Bellman equation :
$$
V(n) = \begin{cases}
A(n) \big[ V(n+N+1) + f_X(n) \big] + B(n) \big[  V(n-1) + S(n) f_Y(n) \big] 
& \text{if} \; 0< n \leq N 
\\
A(n) \big[ V(n+1) + f_X(n) \big] + B(n) \big[  V(n-N) + S(n) f_Y(n) \big]
& \text{if} \; N+1 \leq n < 2N+1 
\end{cases}
$$
or in the matrix form : ($0 <n_1 \leq N$ , $N+1 \leq n < 2N+1$)
$$
\begin{pmatrix}
V(0) \\
\vdots \\
V(n_1) \\
\vdots \\
V(n_2) \\
\vdots \\
V(2N+1)
\end{pmatrix}
=
\begin{pmatrix}
1      &    0    & \ldots &   0   &  \ldots&   0    & 0 \\
\vdots \\
  0    & \ldots  &  \overbrace{B(n)}^{n_1-1\; \text{th column}} & \ldots & 
  \overbrace{A(n)}^{n_1 +N+1 \;\text{th column} } & \ldots & 0 \\ 
\vdots \\
  0    & \ldots  &  \ldots &  \overbrace{B(n)}^{n_2-N\; \text{th column}} &
 \ldots &  \overbrace{A(n)}^{n_2+1 \;\text{th column} } &  0 \\ 
\vdots \\
  0    &    0    &  0    &   0    &  0  &   0     &  1 
\end{pmatrix}
\begin{pmatrix}
V(0) \\
\vdots \\
V(n_1) \\
\vdots \\
V(n_2) \\
\vdots \\
V(2N+1)
\end{pmatrix}
+
\begin{pmatrix}
0\\
\vdots \\
A(n_1)  f_X(n_1) + B(n_1) S(n_1) f_Y(n_1) \\
\vdots \\
A(n_2)  f_X(n_2) + B(n_2) S(n_2) f_Y(n_2) \\
\vdots \\
0
\end{pmatrix}
$$
and the boundary conditions :
$$
V(2N) = W_u \quad , \quad V(1) = W_l
$$

In [2]:
def Matrix(N,A,B):
    Matrix = np.zeros((2*N+2,2*N+2))
   
    # rows 0 ~ 2N+1 
    for n in range(2*N+1):
        #boundary
        if n == 0 :
            Matrix[n,0] = 1

        elif n == 2*N +1 :
            Matrix[n,2*N+1] = 1
        
        
        elif n > 0 and n <= N :
            
            #columns 0 ~ 2N+1
            for m in range(2*N):
                if m == n-1:
                    Matrix[n,m] = B(n)
                elif m== n+N:
                    Matrix[n,m] = A(n)
        elif n > N and n <= 2*N :
            
            #columns 0 ~ 2N+1
            for m in range(2*N):
                if m == n-N:
                    Matrix[n,m] = B(n)
                elif m== n+1:
                    Matrix[n,m] = A(n)
    return Matrix


In [3]:
def V(N,A,B,S,f_X,f_Y):
    Matrix = Matrix(N,A,B) -np.identity(2*N+0)
    vector = - (A * f_X + B * S * f_Y )
    return np.linalg.solve(Matrix , vector)

## Computation of $A(n)$ and $B(n)$

### Formula check
Compute $\mathbb{E}_{(S^*,M)}(e^{-r\tau} \; ; \; S^*_\tau = S^*_0 e^\delta) $
and $\mathbb{E}_{(S^*,M)}(e^{-r\tau} \; ; \; S^*_\tau = S^*_0 e^{-\delta} )$ with $\tau = \{ n \geq 0 \mid S^*_n \neq S^*_0\}$
or equivalently $ \mathbb{E}_{x}(e^{-r\tau} \; ; \; X_\tau =  k+1)$ and $ \mathbb{E}_{x}(e^{-r\tau} \; ; \; X_\tau =(k+1))$  with 
$\tau = \{ n \geq 0 \mid X_n = k+1 \; \text{or} \; -(k+1) \}$ and $x \notin \{ -(k+1) , (k+1)\}$. The idea is to choose $a>0$ s.t $\{ e^{aX_n \; - \; r \tau} \}_{n \geq 0}$ is a martingale w.r.t
$ \{ \mathcal{F}_n \}_{n \geq 0} \equiv \{ \sigma(X_k \; ; \; k \leq n) \}_{ n \geq 0}$  . By some computation we get two solutions:
$$
a_\pm = \log( \frac{e^r \pm \sqrt{e^{2r} -4p(1-p) }}{2p})
$$ 
Using the property of martingale , we can solve the linear equations :
$$
\begin{align*}
 \mathbb{E}_{x}( e^{a_\pm X_0}) = e^{a_\pm x} = \mathbb{E}_{x}( e^{aX_\tau \; - \; r \tau}) 
 = e^{(k+1)a_\pm } \; \mathbb{E}_{x}( e^{aX_\tau \; - \; r \tau}  \; ; \; X_\tau = k+1 ) +
 e^{ -(k+1)a_\pm } \; \mathbb{E}_{x}( e^{aX_\tau \; - \; r \tau}  \; ; \; X_\tau = -k-1 ) 
\end{align*}
$$

In [9]:
# check martingale computation
k = sym.symbols('k')
X = sym.symbols('X')
r = sym.symbols('r')
p = sym.symbols('p')
a = sym.symbols('a')
a_p = sym.log( (sym.exp(r) + sym.sqrt( sym.exp(2*r) -4*p*(1-p) )  )/(2*p)  )
a_m = sym.log( (sym.exp(r) - sym.sqrt( sym.exp(2*r) -4*p*(1-p)  ) )/(2*p)  )
L = sym.exp(-r)*( p*sym.exp(a*(X+1)-r*k ) +  (1-p)*sym.exp(a*(X-1)-r*k )             )
R = sym.exp( a*X - r*k)
#result = Conditional - Exact

In [5]:
sym.simplify(L.subs(a,a_p)/R.subs(a,a_p))-1 , sym.simplify(L.subs(a,a_m)/R.subs(a,a_m))-1

(0, 0)

In [11]:
x = sym.symbols('x')
theta = sym.exp(a*x)
theta_p = theta.subs(a,a_p)
theta_m = theta.subs(a,a_m)
A = (theta_p.subs(x,x+k+1) - theta_m.subs(x,x+k+1) )/(
    theta_p.subs(x,2*(k+1)) - theta_m.subs(x,2*(k+1))
)
B = ( theta_p.subs(x,x-k-1)- theta_m.subs(x,x-k-1) )/(
    theta_p.subs(x,-2*k-2)- theta_m.subs(x,-2*k-2)
)

In [12]:
sym.simplify(A*theta_p.subs(x,k+1) + B*theta_p.subs(x,-k-1) -theta_p) 

0

In [88]:
sym.simplify(A*theta_m.subs(x,k+1) + B*theta_m.subs(x,-k-1) -theta_m)

0

$$
A = \frac{ e^{a_+ ( x+k+1 )} - e^{a_- ( x+k+1 )} }{ e^{a_+ ( 2k+2 )} -  e^{a_- ( 2k+2 )}}
\qquad
B = \frac{ e^{a_+ ( x-k-1 )} - e^{a_- ( x-k-1 )} }{ e^{a_+ ( -2k-2 )} -  e^{a_- ( -2k-2 )}}
$$

### Numerical computation

In [8]:
#return the index of given state
def index(Sp,Sl,Su,M,N,k):
    if M != (k or -k):
        print("only left and right endpoints")
        else:
            if Sp < Sl or Sp > Su:
                
            else:
                print("outside of given boundary")
            
#return the state of given index
def state(n,N,k):
    if n <= N :
        return ( n , -k)
    elif n >= N+1 :
        return (n-N , k)

def a_p(r,p,k):
    return np.log( ( np.exp(r)  np.sqrt(np.exp(2*r) -4*p*(1-p) )  )/(2*p) )
def a_m(r,p,k):
    return np.log( (np.exp(r) - np.sqrt(np.exp(2*r) -4*p*(1-p) ) )/(2*p))
def theta(a,x):
    return np.exp(a*x)

In [28]:
def A(n,N,r,p,k):
    a_p = a_p(r,p,k)
    a_m = a_m(r,p,k)
    if n > 0 and n < 2*N+1 :
        M = state(n,N,k)[1]
        return ( theta(a_p , M+k+1) - theta(a_m , M+k+1) )/(  theta(a_p , 2*k+2 ) - theta(a_m , 2*k+2 ) )
    else:
        return 0
def B(n,N,r,p,k):
    a_p = a_p(r,p,k)
    a_m = a_m(r,p,k)
    if n > 0 and n < 2*N+1 :
        M = state(n,N,k)
        return ( theta(a_p , M-k-1) - theta(a_m , M-k-1) )/(  theta(a_p , -2*k-2 ) - theta(a_m , -2*k-2 ) )
    else:
        return 0

In [ ]:
#fee from X
def f_X(Pa,Pb,L,gamma,n,N,k):
     Sp , M = state(n,N,k)
    
    #state is outside of position range or given boundary
    if Pa > Sp or Pb < = Sp or n== 0 or n==2*N+1:
        return 0
    
    #state is inside of position range
    else:
        
        #state is at left endpoint
        if  n>0 and n <= N :
            Sp_new = np.exp( state( n+N+1 ,N,k)[0] )
        
        #state is at right endpoint
        elif n> N and n <  2*N+1 :
            Sp_new = np.exp( state( n+1 ,N,k)[0] )
        return L*(1-gamma)/gamma *( np.sqrt(  Sp_new  ) - np.sqrt( Sp ))
#fee from Y    
def f_Y(Pa,Pb,L,gamma,n,N,k):
     Sp , M = state(n,N,k)
    
    #state is outside of position range or given boundary
    if Pa >= Sp or Pb < Sp or n== 0 or n==2*N+1:
        return 0
    
    #state is inside of position range
    else:
        #state is at left endpoint
        if  n>0 and n <= N :
            Sp_new = np.exp( state( n-1 ,N,k)[0] )
            
        #state is at right endpoint
        if n> N and n <  2*N+1 :
            Sp_new = np.exp( state( n-N-1 ,N,k)[0] )
        return L*(1-gamma)/gamma *( 1/np.sqrt(  Sp_new  ) - 1/np.sqrt( Sp ))      

In [ ]:
#without scaling assumption
def fee_collecting( P_a,P_b , gamma ,  L , S_p_old , S_p_new , S_m_new ):
    A = max(S_p_old , P_a)
    B = min(S_p_old , P_b)
    fee = 0
    if S_p_old < P_b :
        if S_p_new > A :
            
            fee = L*(1-gamma)/gamma *( np.sqrt( min( S_p_new , P_b) ) - np.sqrt( A ))
            print("forward fee" , fee)
            
    if S_p_old > P_a :
        if S_p_new < B :
            fee = L*(1-gamma)/gamma *S_m_new*( 1/np.sqrt( max( S_p_new , P_a) ) - 1/np.sqrt( B ))
            print("backward fee" , fee)
    return fee